# **NL booster_coverage_archived_20220904**


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

TABLE

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSARCHIVE].[RIVM_BOOSTER_SHOT_COVERAGES_NL_ARCHIVED_20220904]'))
CREATE TABLE [VWSARCHIVE].[RIVM_BOOSTER_SHOT_COVERAGES_NL_ARCHIVED_20220904] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [DATE_OF_STATISTICS] DATETIME,
    [DATE_OF_REPORT] DATETIME,
	[AGE_GROUP] VARCHAR(50) NULL,
    [BIRTH_YEAR] VARCHAR(50),
    [PERCENTAGE] FLOAT
);
GO

STORED PROCEDURE

In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_BOOSTER_SHOT_COVERAGES_NL_ARCHIVED_20220904] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[RIVM_BOOSTER_SHOT_COVERAGES_NL_ARCHIVED_20220904]

    INSERT INTO [VWSARCHIVE].[RIVM_BOOSTER_SHOT_COVERAGES_NL_ARCHIVED_20220904] (
        [DATE_LAST_INSERTED],
        [DATE_OF_STATISTICS],
        [DATE_OF_REPORT],
        [AGE_GROUP],
        [BIRTH_YEAR],
        [PERCENTAGE]
    )
    SELECT 
        [DATE_LAST_INSERTED],
        [DATE_OF_STATISTICS],
        [DATE_OF_REPORT],
        [AGE_GROUP],
        [BIRTH_YEAR],
        [PERCENTAGE]
    FROM [VWSDEST].[RIVM_BOOSTER_SHOT_COVERAGES_NL]
    WHERE [DATE_LAST_INSERTED] >= '2022-09-07' AND [DATE_LAST_INSERTED] < '2022-09-08' -- get first insertion after 20220904
END


VIEW

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_BOOSTER_SHOT_COVERAGES_NL_ARCHIVED_20220904] AS
    WITH CTE AS (
        SELECT
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS]) AS [DATE_UNIX],            
            -- UPPER(DATENAME(dw, [DATE_OF_STATISTICS])) AS [DAY_OF_WEEK],
            [AGE_GROUP],
            CAST([PERCENTAGE] AS [NUMERIC](10,1)) AS [PERCENTAGE]
        FROM [VWSARCHIVE].[RIVM_BOOSTER_SHOT_COVERAGES_NL_ARCHIVED_20220904] WITH(NOLOCK)
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSARCHIVE].[RIVM_BOOSTER_SHOT_COVERAGES_NL_ARCHIVED_20220904] WITH(NOLOCK))
            AND [AGE_GROUP] IN ('18+', '12+')
    )
    SELECT
        [AGE_GROUP],
        [PERCENTAGE],
        [DATE_OF_INSERTION_UNIX],
        [DATE_UNIX]
    FROM CTE
    WHERE [DATE_UNIX] = (
        SELECT 
            MAX([DATE_UNIX]) 
        FROM CTE 
        -- WHERE [DAY_OF_WEEK] = 'SUNDAY'
    )
GO

PROTOS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_BOOSTER_SHOT_COVERAGES_NL_ARCHIVED_20220904',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'booster_coverage_archived_20220904',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 2,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO